##Instalacion de dependecias e importar librerias


In [1]:
#!pip install tensorflow-gpu

import datetime
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import  Conv2D,Conv2DTranspose,MaxPooling2D, Flatten, Input, Dense,Reshape
from keras.models import Model
from keras.optimizers import adam
from keras.callbacks import TensorBoard
from keras.utils import Sequence
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from IPython.display import clear_output
import cv2
import numpy as np


  Could not find a version that satisfies the requirement tensorflow (from versions: )
No matching distribution found for tensorflow
You are using pip version 19.0.2, however version 19.2.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
Using TensorFlow backend.


ImportError: No module named 'tensorflow'

##Definicion de Funciones y clases

In [0]:
##Funciones para guardar imagenes a TensorBoard
##---------------------------------------------

from PIL import Image
import io
video = [] #Arreglo para guardar frames que se convertiran en video

def make_image(tensor):

    tensor = (tensor*255).astype(np.uint8)
    print(tensor.shape)
    height, width, channel = tensor.shape
    image = Image.fromarray(tensor)
    output = io.BytesIO()
    image.save(output, format='PNG')
    image_string = output.getvalue()
    output.close()

    return tf.Summary.Image(height=height,
                            width=width,
                            colorspace=channel,
                            encoded_image_string=image_string)
  
class TensorBoardImage(keras.callbacks.Callback): ##Callback para guardar imagen de prueba

    def __init__(self, tag, log_dir):
        super().__init__() 
        self.tag = tag
        self.log_dir = log_dir

    def on_epoch_end(self, epoch, logs={}):

        img = self.model.predict(val_gen.__getitem__(0)[0])
        image = make_image(img[0])
        video.append(img[0])                 #---------Agrega fotograma al video
        summary = tf.Summary(value=[tf.Summary.Value(tag=self.tag, image=image)])
        dirc = "./logs/imagen/" + self.tag
        writer = tf.summary.FileWriter(dirc)
        writer.add_summary(summary, epoch)
        writer.close()
        return

##Generador Modificado entrenamiento
##----------------------------------
class Generador(Sequence):
  
  def __init__(self, genoriginal):
    self.gene = genoriginal
    
  def __len__(self):
    return int(self.gene.n/self.gene.batch_size)
  
  def __getitem__(self, x):
    b = self.gene.batch_size
    cat= self.gene.__getitem__(x)[1]
    items = self.gene.__getitem__(x)[0]
    x = []
    for item in items:
      x.append(item)
    return (np.array(x), {"cla4": cat, "dec3": items})
    
  def on_epoch_end(self):
    print("Final de epoca")

##Inicio del servidor TensorBoard y tunnel Ngrok
##----------------------------------------------

LOG_DIR = './logs/'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 --reload_interval 5 &'.format(LOG_DIR)
)

##Definicion de imagenes de entrenamiento

In [0]:
train_path = "/dataset/entrenamiento/"
val_path = "/dataset/validacion/"
train_gen = ImageDataGenerator(data_format="channels_last",rescale=1/255).flow_from_directory(train_path, target_size=(256,256),batch_size=10, class_mode="binary")
val_gen = ImageDataGenerator(data_format="channels_last",rescale=1/255).flow_from_directory(val_path, target_size=(256,256),batch_size=2, class_mode="binary")

##Definicion de los modelos

In [0]:
#----Entrada----------------------
entrada = Input(shape=(256,256,3), name="entrada")
#---------------------------------

layers_to_freeze = ["enc1","enc2","enc3","dec1","dec2","dec3", "cla1", "cla2", "cla3", "cla4"] # Lista de capas que se congelaran 

#Definir capas----------------------
#-----------------------------------
enc1 = Conv2D(filters=32, kernel_size=(5,5), activation="relu", name="enc1")
enc2 = Conv2D(filters=32, kernel_size=(4,4), activation="relu", name="enc2")
enc3 = Conv2D(filters=64, kernel_size=(4,4), activation="relu", name="enc3")

dec1 = Conv2DTranspose(filters=64, kernel_size=(4,4), activation="relu", name="dec1")
dec2 = Conv2DTranspose(filters=32, kernel_size=(4,4), activation="relu", name="dec2")
dec3 = Conv2DTranspose(filters=3, kernel_size=(5,5), activation="sigmoid", name="dec3")

cla1 = Dense(units=256, activation="relu", name="cla1")
cla2 = Dense(units=256, activation="relu", name="cla2")
cla3 = Dense(units=256, activation="relu", name="cla3")
cla4 = Dense(units=1, activation="sigmoid", name="cla4")

##Modelo1------------------------------------
##-------------------------------------------

m11 = enc1(entrada)
m12 = enc2(m11)
m13 = enc3(m12)
m14 = Flatten()(m13)
m15 = Reshape((246,246,64))(m14)
m16 = dec1(m15)
m17 = dec2(m16)
m18 = dec3(m17)
f2 = Flatten()(m18)
m19 = cla1(f2)
m20 = cla2(m19)
m21 = cla3(m20)
m22 = cla4(m21)

modelo1 = Model(inputs=entrada, outputs=[m18])
modelo1.compile(loss=["logcosh"],optimizer=adam(lr=0.00001, decay=0.00001),metrics=["accuracy"])

##Modelo2------------------------------------
##-------------------------------------------

n11 = enc1(entrada)
n12 = enc2(n11)
n13 = enc3(n12)
n14 = Flatten()(n13)
fake1 = Dense(units=256, activation="relu")(n14)
fake2 = Dense(units=256, activation="relu")(fake1)
fake2 = Dense(units=256, activation="relu")(fake1)
fake2 = Dense(units=256, activation="relu")(fake1)
fake3 = Dense(units=3873024, activation="sigmoid")(fake2)
n15 = Reshape((246,246,64))(fake3)
n16 = dec1(n15)
n17 = dec2(n16)
n18 = dec3(n17)

modelo2 = Model(inputs=entrada, outputs=[n18])
modelo2.compile(loss=["logcosh"],optimizer=adam(lr=0.0001, decay=0.0001),metrics=["accuracy"])

##----Clasificador---------------------------
##-------------------------------------------

entradaC = Input(shape=(256,256,3), name="entradaC")
f3 = Flatten()(entradaC)
n19 = cla1(f3)
n20 = cla2(n19)
n21 = cla3(n20)
n22 = cla4(n21)

clasificador = Model(inputs=entradaC, outputs=[n22])
clasificador.compile(loss=["binary_crossentropy"], optimizer=adam(lr=0.0001, decay=0.0001),metrics=["accuracy"])

#------------------------------------------------
#------------------------------------------------
##--Funciones para congelar y descongelar pesos

def congelar_pesos(modelo):
  for layer in modelo.layers:
    if layer.name in layers_to_freeze:
      layer.trainable = False

def descongelar_pesos(modelo):
  for layer in modelo.layers:
    if layer.name in layers_to_freeze:
      layer.trainable = True

##------------------------------------------------
##------------------------------------------------

In [0]:
clasificador.summary()


##Entrenamiento modelo


In [0]:
logDir = "./logs/escalar/"+ str(datetime.datetime.now().time()) #Directorio de logs
nameImagen = str(datetime.datetime.now().time()) #Nombre de la imagen predicha por el modelo

tbimg= TensorBoardImage(tag=nameImagen, log_dir= logDir)  ##Callback que escribe imagen de prediccion del modelo
tb = TensorBoard(log_dir=logDir, histogram_freq=0,write_graph=True,write_grads=True,write_images=True) ##Callback de Loss y accuracy

gen = Generador(train_gen) #Iniciacion del Generador modificado

modelo1.fit_generator(generator = gen, steps_per_epoch=1,epochs=100,shuffle=True, callbacks=[tb,tbimg])

##Generar video de muestra


In [0]:
im = val_gen.__getitem__(0)[0]
m = modelo1.predict(im)
print(m[0].shape)
#plt.imshow((img[0]*255).astype(np.uint8))

In [0]:
# def entrenar(epocas,nrandom):
#   i= 0
#   for b in train_batch:
#     i+=1
#     if i >= epocas:

#       #print("MOD")
#       modelo.metrics_names, modelo.train_on_batch(x=b[0],y=b[0])
#       im =modelo.predict(val_batch[nrandom][0])
#       #plt.imshow(im[0])
#       #plt.show()
#       return im[0]
#       break

In [0]:
# fourcc = cv2.VideoWriter_fourcc(*"XVID")
# video = cv2.VideoWriter('/video{}.avi'.format(nrandom), fourcc, 30, (256, 256), True)
# for j in img:
#   #print(np.uint8(j*255))
#   video.write(np.uint8(j*100))
# video.release()
